# Analysis of call data

In [18]:
import avro.schema
from avro.datafile import DataFileReader
from avro.io import DatumReader
import csv
from flatten_dict import flatten
import re
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

### Get the input data and extract AVRO schema

In [19]:
inputFile = 'CT_Sample.avro'
reader = DataFileReader(open(inputFile,"rb"),DatumReader())
schema = reader.meta
input_data_schema = schema['avro.schema'].decode('latin1')

In [29]:
input_data_schema

'{"type":"record","name":"topLevelRecord","fields":[{"name":"DusTimestamp","type":["string","null"]},{"name":"Version","type":["string","null"]},{"name":"CtId","type":["string","null"]},{"name":"TimeInfo","type":[{"type":"record","name":"TimeInfo","fields":[{"name":"TimestampTableau","type":["string","null"]},{"name":"TimestampDateTime","type":["string","null"]},{"name":"TimestampOffset","type":["double","null"]},{"name":"TimestampMillis","type":["long","null"]},{"name":"IsSynced","type":["boolean","null"]},{"name":"TimeSource","type":["string","null"]},{"name":"MillisSinceLastSync","type":["long","null"]},{"name":"DeviceDriftMillis","type":["long","null"]}]},"null"]},{"name":"TestTimestamp","type":["string","null"]},{"name":"ScreenState","type":["string","null"]},{"name":"ServerFilename","type":["string","null"]},{"name":"DurationTcpConnect","type":["long","null"]},{"name":"DurationHttpReceive","type":["long","null"]},{"name":"BytesRead","type":["long","null"]},{"name":"HeaderBytesRea

### Flatten the data using '_' separator, compute average and max RxLevel

In [21]:
def underscore_partitioner(key1, key2):
    if key1 is None:
        return key2
    else:
        return key1 + "_" + key2

In [22]:
csv_columns = set()
flattenedData = list()
i = 0
sum = 0
max_RxLevel = None
average_RxLevel = None

for topLevelRecord in reader:
    csv_columns = csv_columns.union(set(flatten(topLevelRecord, reducer=underscore_partitioner).keys()))
    flattenedData.append(flatten(topLevelRecord, reducer=underscore_partitioner))
    if (topLevelRecord['Simoperator'] == '26202'):
        sum = sum + topLevelRecord['RxLevel']
        if (i == 0):
            max_RxLevel = topLevelRecord['RxLevel']
        elif (topLevelRecord['RxLevel'] > max_RxLevel):
            max_RxLevel = topLevelRecord['RxLevel']        
        i+=1
        
if (sum != 0 and i != 0):
    average_RxLevel = sum / i

In [23]:
max_RxLevel

2147483647

In [24]:
average_RxLevel

21261053.49066633

### Write the content into a csv file

In [25]:
outputCSVFile = re.sub(r'\..+','',inputFile) + '.csv'
try:
    with open(outputCSVFile, 'w') as file:
        csv_writer = csv.DictWriter(file, fieldnames=list(csv_columns))
        csv_writer.writeheader()
        csv_writer.writerows(flattenedData)
except IOError:
    print("I/O error") 

### Write the content into parquet file with partition as first 3 characters of 'Simoperator' field

In [26]:
outputParquetFile = re.sub(r'\..+','',inputFile)# + '.parquet'
df_call_data = pd.read_csv(outputCSVFile)
df_call_data['SimOperator_key'] = df_call_data['Simoperator'].apply(lambda x : str(x)[:3])
df_call_data.head()

,TimeInfo_TimestampMillis,BatteryInfo_BatteryChargePlug,BatteryInfo_BatteryVoltage,ServerFilename,TestTimestamp,DurationHttpGetCommand,DurationSSL,CtId,StorageInfo_StorageExternalAudio,StorageInfo_StorageExternalVideo,...,StorageInfo_StorageInternalVideo,IdleStateOnEnd,StorageInfo_StorageInternalImages,Simoperator,DurationTcpConnect,BatteryInfo_BatteryRemainingEnergy,BatteryInfo_MissingPermission,StorageInfo_StorageInternalAvailable,TimeInfo_DeviceDriftMillis,SimOperator_key
0,1558273271827,AC,3988,/favicon.ico?id=-4673789195750544260,2019-05-19 15:41:11.827 +0200,-1,-1,3babf458ad66c6fde96b5b74153f555f44506f83c30db4...,207196065,3139611762,...,0,NonIdle,0,26202,-1,0,False,4148498432,0,262
1,1558275124347,Unknown,3734,/favicon.ico?id=2590331085222981885,2019-05-19 16:12:04.347 +0200,-1,-1,707f3e1461a541b9153ae4731728aced646330faaabdb9...,207196065,3139611762,...,0,NonIdle,0,26202,-1,0,False,4143808512,0,262
2,1558276638106,Unknown,3714,/favicon.ico?id=-5915931742619868906,2019-05-19 16:37:18.106 +0200,-1,-1,4218833d296e4dfa1d0b7ae2f31afc5c54d81ecdb11d42...,207196065,3139611762,...,0,NonIdle,0,26202,-1,0,False,4146425856,0,262
3,1558328619292,Unknown,4006,/favicon.ico?id=1322144946858327674,2019-05-20 07:03:39.292 +0200,-1,-1,0306c6d2c83788ef77c789f7daa56bfcca2e8dfc769edf...,207196065,3139611762,...,0,NonIdle,0,26202,-1,0,False,4099047424,0,262
4,1558255958392,Unknown,4043,/favicon.ico?id=2543913046274584870,2019-05-19 10:52:38.392 +0200,0,157,765ac7cbc1e7cebe35f56d62888cebd85886977582a82a...,27847736,12646488195,...,0,NonIdle,0,26201,33,0,False,1193091072,-26,262


In [27]:
df_call_data_table = pa.Table.from_pandas(df_call_data, preserve_index=False)
#pq.write_table(df_call_data_table, outputParquetFile)
pq.write_to_dataset(df_call_data_table, root_path=outputParquetFile, partition_cols=['SimOperator_key'])

### Read from the saved parquet file into pandas dataframe

In [28]:
pq.read_table(outputParquetFile).to_pandas().drop(columns='SimOperator_key')

,TimeInfo_TimestampMillis,BatteryInfo_BatteryChargePlug,BatteryInfo_BatteryVoltage,ServerFilename,TestTimestamp,DurationHttpGetCommand,DurationSSL,CtId,StorageInfo_StorageExternalAudio,StorageInfo_StorageExternalVideo,...,IdleStateOnStart,StorageInfo_StorageInternalVideo,IdleStateOnEnd,StorageInfo_StorageInternalImages,Simoperator,DurationTcpConnect,BatteryInfo_BatteryRemainingEnergy,BatteryInfo_MissingPermission,StorageInfo_StorageInternalAvailable,TimeInfo_DeviceDriftMillis
0,1558084729640,Unknown,4058,/favicon.ico?id=5072324220505691235,2019-05-17 11:18:49.640 +0200,1,109,dcd43979d35f58c51785db2aafff8231d521dc47ac12fe...,990333398,3529067430,...,LightIdle,0,NonIdle,0,23101,69291,0,False,37718720512,69
1,1558096570052,Unknown,4060,/favicon.ico?id=2464754380675024255,2019-05-17 14:36:10.052 +0200,-1,-1,3e41a43ecefe4888ba6c0f018ef06ec71fa92d14b8d086...,990333398,3529067430,...,NonIdle,0,NonIdle,0,23101,-1,0,False,37497860096,0
2,1558114482761,Unknown,3796,/favicon.ico?id=8039176397785528942,2019-05-17 19:34:42.761 +0200,-1,-1,bb3e08722ea31f4eb725d2bdb97f8aa2b62b1c9672ab99...,990333398,3529067430,...,LightIdle,0,LightIdle,0,23101,-1,0,False,37514944512,0
3,1558170672443,Unknown,3978,/favicon.ico?id=-5451822870982144652,2019-05-18 11:11:12.443 +0200,-1,-1,5093b8274832f0f53efad5009571b97529bdf07058481f...,990333398,3529067430,...,NonIdle,0,LightIdle,0,23101,-1,0,False,37678772224,-812
4,1558260263145,Unknown,3708,/favicon.ico?id=2557866456466301923,2019-05-19 12:04:23.145 +0200,-1,-1,4d219b644c81e523c760795441a570719d9139b9088c8d...,774319439,10496825540,...,NonIdle,0,NonIdle,0,23106,-1,0,False,16226443264,605
5,1558262305612,Unknown,3799,/favicon.ico?id=-2324992792131232887,2019-05-19 12:38:25.612 +0200,-1,-1,93cafeefb5925c615493e41274a7c05af0458d98326424...,774319439,10496825540,...,NonIdle,0,NonIdle,0,23106,-1,0,False,16219807744,604
6,1558263862712,Unknown,3759,/favicon.ico?id=2443499960882418763,2019-05-19 13:04:22.712 +0200,-1,-1,f1456309394c21893d27a204aaac6589a46869cd202af9...,774319439,10496825540,...,NonIdle,0,NonIdle,0,23106,-1,0,False,16210190336,605
7,1558265700909,Unknown,3680,/favicon.ico?id=423154583146001676,2019-05-19 13:35:00.909 +0200,-1,-1,7aec18c9ac5d8dea1a4b8d8881168ced98a4e6a4bf499f...,774319439,10496825540,...,NonIdle,0,NonIdle,0,23106,-1,0,False,16174874624,604
8,1558267440131,USB,3902,/favicon.ico?id=-9015608293294523455,2019-05-19 14:04:00.131 +0200,3,1382,d15bf2ab5b960b1ea5d1d339760d563dc1fefcc332245e...,774319439,10496825540,...,NonIdle,0,NonIdle,0,23106,251,0,False,16166199296,604
9,1558268282146,Unknown,3793,/favicon.ico?id=-8170255703418532939,2019-05-19 14:18:02.146 +0200,-1,-1,041311e3e5fbd1392867cf243da2592cb87935da24a2e8...,774319439,10496825540,...,LightIdle,0,NonIdle,0,23106,-1,0,False,16169209856,605
